### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [46]:
#Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path


In [47]:
os.getcwd()

'/Users/harshul/website clone/harshul/test/groupwebsite_generator/notebooks'

In [48]:
current_dir = Path(os.getcwd())

MEMBERS_DIR_PATH = current_dir / '../../group-data/members/'
WEBSITE_DATA_PATH = current_dir / '../../group-data/website_data/'
CONTENT_DIR_PATH = current_dir / '../../group-data/website_data/content/'


In [49]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [50]:
# For people page
def extract_member_data(member_dir):
    if member_dir.name.startswith('.'):
        return None

    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    
    top_experience_role = ""
    top_education_degree = ""
    top_education_end_date = None
  
    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
        if experiences:
            top_experience_role = experiences[0].get('role', "")
    
    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))
        if education:
            top_education_degree = education[0].get('degree', "")
            top_education_end_date = education[0].get('end_date')
    
    if (member_projects := (member_json_dir / 'projects.json')).exists():
        current_position = json.load(open(member_projects))[0]
        member_record.update(current_position)
    
    if (member_links := (member_json_dir / 'social_links.json')).exists():
        social_links = json.load(open(member_links))
        member_record.update(social_links)

    return member_record, top_experience_role, top_education_degree, top_education_end_date


def assign_academic_role(top_experience_role, top_education_degree, top_education_end_date):
    if 'Assistant Professor' in top_experience_role or 'Professor' in top_experience_role:
        return 'Professor'
    elif top_experience_role in ['Visiting Researcher', 'Postdoctoral Researcher']:
        return 'Postdoctoral researcher'
    elif top_education_degree == 'Bachelors':
        return 'Graduate Student' if top_education_end_date is not None else 'Undergraduate Student'
    elif top_education_degree in ['Masters', 'PhD']:
        return 'Graduate Student'
    else:
        return 'Other'

member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    extracted_data = extract_member_data(member_dir)
    if extracted_data:
        member_record, top_experience_role, top_education_degree, top_education_end_date = extracted_data
        academic_role = assign_academic_role(top_experience_role, top_education_degree, top_education_end_date)


        nick_name = member_record.get('nick_name', None)
        second_name = member_record.get('last_name', "")
        if not nick_name:
            first_name = member_record.get('first_name', "")
            last_name = member_record.get('last_name', "")
            name = f"{first_name} {last_name}"
        else:
            name = f"{nick_name} {second_name}"


        member_records.append({"name": name, "academic_role": academic_role})


people_page_df = pd.DataFrame(member_records)

In [51]:
# To view dict structure.
people_page_df.to_dict(orient='records')

[{'name': 'Josh Shields', 'academic_role': 'Graduate Student'},
 {'name': 'Anirban  Dutta', 'academic_role': 'Postdoctoral researcher'},
 {'name': 'Erin Visser', 'academic_role': 'Undergraduate Student'},
 {'name': 'Vicente  Amado Olivo', 'academic_role': 'Graduate Student'},
 {'name': 'Yuki Matsumura', 'academic_role': 'Graduate Student'},
 {'name': 'Andrew Fullard', 'academic_role': 'Graduate Student'},
 {'name': 'Isaac Smith', 'academic_role': 'Graduate Student'},
 {'name': 'Hayden Monk', 'academic_role': 'Undergraduate Student'},
 {'name': 'Atharva Arya', 'academic_role': 'Graduate Student'},
 {'name': 'Tripp Dow', 'academic_role': 'Undergraduate Student'},
 {'name': 'Bea Lu', 'academic_role': 'Undergraduate Student'},
 {'name': 'Sona Chitchyan', 'academic_role': 'Other'},
 {'name': 'Iliomar Rodriguez-Ramos', 'academic_role': 'Undergraduate Student'},
 {'name': 'Jaladh Singhal', 'academic_role': 'Graduate Student'},
 {'name': "Jack O'Brien", 'academic_role': 'Graduate Student'},
 {

In [52]:
# To view dataframe
people_page_df


,name,academic_role
0,Josh Shields,Graduate Student
1,Anirban Dutta,Postdoctoral researcher
2,Erin Visser,Undergraduate Student
3,Vicente Amado Olivo,Graduate Student
4,Yuki Matsumura,Graduate Student
5,Andrew Fullard,Graduate Student
6,Isaac Smith,Graduate Student
7,Hayden Monk,Undergraduate Student
8,Atharva Arya,Graduate Student
9,Tripp Dow,Undergraduate Student


In [53]:



website_data_files = {
    'contact': 'contact.json',
    'general': 'general.json',
    'research': 'research.json',
    'homepage': 'homepage.json',
    'support': 'support.json'
}


contact, general, research, homepage, support = [{}] * 5


for var_name, webpage_json in website_data_files.items():
    file_path = WEBSITE_DATA_PATH / webpage_json
    if file_path.exists():
        with open(file_path, 'r') as f:
            locals()[var_name] = json.load(f)


In [54]:
contact_df = pd.DataFrame.from_records([contact])
general_df = pd.DataFrame.from_records([general])
homepage_df = pd.DataFrame.from_records([homepage])
research_df = pd.DataFrame.from_records([research])
support_df = pd.DataFrame.from_records([support])


In [55]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'),extensions=['jinja2.ext.loopcontrols'])
environment.globals['page_link'] = page_link


In [56]:
general_dict = general_df.iloc[0].to_dict()
homepage_dict = homepage_df.iloc[0].to_dict()


##### People Page

In [57]:
templates_dir_path = current_dir / '../templates/'
environment = Environment(loader=FileSystemLoader(str(templates_dir_path)))
people_template = environment.get_template("people.html.j2")


In [58]:
people_template = environment.get_template("people.html.j2")

In [59]:
environment.globals['page_link'] = page_link
people_content = people_template.render(general=general_dict, present_members=people_page_df, result=result)


NameError: name 'result' is not defined

In [ ]:
with open("../../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)